In [22]:
import pickle

# text to csv

In [1]:
f = open("Spam SMS Collection.txt", "r")

In [2]:
new=[]
for i in f:
    a= i.split("\t")
    new.append(a)

In [3]:
new

[['ham',
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n'],
 ['ham', 'Ok lar... Joking wif u oni...\n'],
 ['spam',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n"],
 ['ham', 'U dun say so early hor... U c already then say...\n'],
 ['ham', "Nah I don't think he goes to usf, he lives around here though\n"],
 ['spam',
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv\n"],
 ['ham',
  'Even my brother is not like to speak with me. They treat me like aids patent.\n'],
 ['ham',
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune\n"],
 ['spam',
  'WINNER!! As a valued network customer you have been selec

In [4]:
import pandas as pd

In [5]:
df= pd.DataFrame(new)

In [6]:
df

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...\n
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will Ã¼ b going to esplanade fr home?\n
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [7]:
df.to_csv('data.csv')

# Model building

In [8]:
import pandas as pd

In [9]:
df= pd.read_csv('data.csv')

In [10]:
df.drop('Unnamed: 0', axis=1, inplace= True)

In [11]:
df.columns= ['target', 'message']

In [12]:
df

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...\n
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will Ã¼ b going to esplanade fr home?\n
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [13]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to C:\Users\Krishna
[nltk_data]     Dalal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Cleaning the messages
corpus= []
ps= PorterStemmer()

for i in range(0,df.shape[0]):

    # Cleaning special character from the message
    message= re.sub(pattern='[^a-zA-Z]', repl=' ', string=df.message[i])

    # Converting the entire message into lower case
    message= message.lower()
    message= message.strip()

    # Tokenizing the review by words
    words= message.split()

    # Removing the stop words
    words= [word for word in words if word not in set(stopwords.words('english'))]

    # Stemming the words
    words= [ps.stem(word) for word in words]

    # Joining the stemmed words
    message= ' '.join(words)

    # Building a corpus of messages
    corpus.append(message)
print(words)

['rofl', 'true', 'name']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec= CountVectorizer()
X= count_vec.fit_transform(corpus).toarray()

In [24]:
pickle.dump(count_vec, open('count_vector.pkl', 'wb'))

In [16]:
y= pd.get_dummies(df.target, drop_first= True)
y[:5]

,spam
0,0
1,0
2,1
3,0
4,0


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, stratify= y, random_state= 10)

In [18]:
from sklearn import naive_bayes
clf = naive_bayes.MultinomialNB()
model= clf.fit(X_train, y_train)

C:\Users\Krishna Dalal\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [19]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(y_train,model.predict(X_train))

0.9925992374971967

In [21]:
accuracy_score(y_test,model.predict(X_test))

0.97847533632287

In [23]:
filename = 'spam_ham_model.pkl'
pickle.dump(model, open(filename, 'wb'))